In [7]:
import tensorflow as tf
import numpy as np
import pandas as pd
import tflearn

In [8]:
raw_train = pd.read_csv('../MNIST/input/train.csv')
raw_test = pd.read_csv('../MNIST/input/test.csv')

In [9]:
def to_one_hot(label):
    base = np.zeros([label.shape[0],10])
    base[np.arange(label.shape[0]),label[:,0].tolist()] = 1
    return base

In [10]:
def next_batch(num, train, label):
    idx = np.arange(0 , len(train))
    np.random.shuffle(idx)
    idx=idx[:num]
    data_shuffle = train[idx,:]
    label_shuffle = label[idx,:]
    return data_shuffle, label_shuffle

In [11]:
train_255 = raw_train.iloc[:,1:].values
raw_label = raw_train.iloc[:,0].values.reshape([train_255.shape[0],1])
label = to_one_hot(raw_label)
train = train_255.astype(np.float)/255.0
print(train.shape)
print(label.shape)
test_255 = raw_test.values
test = test_255.astype(np.float)/255.0
print(test.shape)

(42000, 784)
(42000, 10)
(28000, 784)


In [17]:
learning_rate = 0.01
training_epochs = 100
batch_size = 100

tf.reset_default_graph()

# Inputs
net = tflearn.input_data([None, 784])

# Hidden layer(s)
net = tflearn.fully_connected(net, 128, activation='ReLU')
net = tflearn.fully_connected(net, 32, activation='ReLU')

# Output layer and training model
net = tflearn.fully_connected(net, 10, activation='softmax')
net = tflearn.regression(net, optimizer='sgd', learning_rate=learning_rate, loss='categorical_crossentropy')

model = tflearn.DNN(net)

# Training
model.fit(train, label, validation_set=0.1, show_metric=True, batch_size=batch_size, n_epoch=training_epochs)

Training Step: 37799  | total loss: 0.07537 | time: 1.555s
| SGD | epoch: 100 | loss: 0.07537 - acc: 0.9817 -- iter: 37700/37800
Training Step: 37800  | total loss: 0.07810 | time: 2.582s
| SGD | epoch: 100 | loss: 0.07810 - acc: 0.9825 | val_loss: 0.14832 - val_acc: 0.9569 -- iter: 37800/37800
--


In [24]:
result = np.array(model.predict(test)).argmax(axis=1)
result

array([2, 0, 9, ..., 3, 9, 2])

In [25]:
dtype = [('ImageId','int32'), ('Label','int32')]
values = np.zeros(test.shape[0], dtype=dtype)
index = np.arange(1,test.shape[0]+1)
df = pd.DataFrame(values, index=index)
df['ImageId'] = index
df['Label'] = result
df.to_csv('./input/submission_dnn.csv', sep=',', encoding='utf-8', index=False)